In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
!{sys.executable} -m pip install "dask[complete]"

In [ ]:
import sys
!{sys.executable} -m pip install dask-ml[complete]

In [ ]:
#Dagshub library installation
import sys
!{sys.executable} -m pip install dagshub mlflow

In [ ]:

#Initialization of mlflow in dagshub
import dagshub
dagshub.init(repo_owner='mirsakhawathossain', repo_name='Asteroid_ML', mlflow=True)

In [ ]:
#Import mlflow
import mlflow

In [ ]:
import pandas as pd

In [ ]:
#Enable Autolog
mlflow.autolog()

In [ ]:
df_feature = pd.read_csv('https://dagshub.com/mirsakhawathossain/Asteroid_ML/raw/72e6f7d1fe31ca3ed5a44e975a4bc4f2ebf4b675/Data/final_data.csv').sort_index(axis=0)

In [ ]:
df_target = pd.read_csv('https://dagshub.com/mirsakhawathossain/Asteroid_ML/raw/13ee6a319985334bbe62daa1985ceaecefae541e/Data/pha_map.csv').sort_index(axis=0)

In [ ]:
df_feature_drop = df_feature.drop(columns=['albedo','rot_per','e','a','q','om','w','ma','ad'])

In [ ]:
from dask.distributed import Client
#import joblib

client = Client(n_workers=32)
client

In [ ]:
import dask.dataframe as dd

In [ ]:
X_dask = dd.from_pandas(df_feature_drop, npartitions=100)
X_dask = X_dask.persist()
y_dask = dd.from_pandas(df_target, npartitions=100)
y_dask = y_dask.persist()

In [ ]:
from dask_ml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_dask, y_dask, test_size=0.4, random_state=42, shuffle=True)

In [ ]:
import dask
X_train, X_test, y_train, y_test = dask.persist(X_train, X_test, y_train, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [ ]:
max_depth = list(range(150, 200,10))
min_samples_split = list(range(12, 20, 5))
min_samples_leaf = list(range(5, 10, 5))
max_features = ['sqrt', 'log2', None]
criterion = ['gini', 'entropy']
n_estimators = list(range(400, 500, 50))

In [ ]:
param_grid = dict(max_depth=max_depth,min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf,max_features=max_features,criterion=criterion,n_estimators=n_estimators)

In [ ]:
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=10,random_state=10)

In [ ]:
rf_classifier = RandomForestClassifier(random_state=43)

In [ ]:
import dask_ml.model_selection as dcv
hyp_se = dcv.HyperbandSearchCV(estimator=rf_classifier,parameters=param_grid,n_iter=10,scoring='accuracy',cv=cv,random_state=11,n_jobs=-1)

In [ ]:
hyp_se.fit(X_train_rb,y_train)

In [ ]:
# summarize result
print('Best Score: %s' %hyp_se.best_score_)
print('Best Hyperparameters: %s' %hyp_se.best_params_)
print('Best Estimator: %s' %hyp_se.best_estimator_)